## Imports and declerations

In [10]:
import pickle
import os
import string
import re
import pprint
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import boolean
import pyparsing
import copy
from pyparsing import Word, alphas, oneOf, operatorPrecedence, opAssoc
InteractiveShell.ast_node_interactivity = "all"



os.getcwd()

'C:\\Users\\hassa\\Desktop\\Development\\Uni Projects\\Information Retrieval\\A1'

## Raw Vocabulary Storage:

In [11]:
from nltk.stem import PorterStemmer

In [12]:

# Break words like Veterans.Before, West.In amendment.Change

def split_words(vocabl):
    new_vocab = set()
    for word in vocabl:
        if re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word) is not None:
            print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split('.')
#             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        elif re.search('^[a-zA-Z]+[?][a-zA-Z]+$',word) is not None:
# #             print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split('?')
# #             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        elif re.search('^[a-zA-Z]+[,][a-zA-Z]+$',word) is not None:
# #             print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split(',')
# #             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        else:
            new_vocab.add(word)
    return new_vocab

# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))


In [40]:
class PostingList(object):
    def __init__(self):
        self.total_count = 0
        self.token = ''
        self.occurrance = {
#             'doc_id':0 = 'positions' : [],
#             
        }
         
    def __repr__(self):
        
        return f'total_cnt : {self.total_count} docs : [{self.occurrance.keys()}]'
    
    def addOccurrance(self, doc_id, position):
        self.total_count += 1
#         print(position)
        if doc_id not in self.occurrance.keys():
            self.occurrance[doc_id] = []
        self.occurrance[doc_id].append(position)
#         self.occurrance[doc_id]['position'].append(pos)
    
class InvertedIndex(object):
    
    def __init__(self):
        self.index = {}
        self.docs = {}
    
    def __len__(self):
        return len(self.index.keys())

In [102]:
vocab = set()
doc_contents = []
inverted_index = InvertedIndex()
printable = set(string.printable) 
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


ps = PorterStemmer()

stop_words = set()
with open('Stopword-List.txt', 'r') as stop_word_file:
    lines = stop_word_file.readlines()
    for line in lines:
        stop_words.add(line.split('\n')[0])
    stop_words.remove('')

    
    
for file_number in range(0, 56):
    with open(f'data/Trump Speechs/speech_{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
#         print(f'File Number : speech_{file_number}.txt' )
#         print(lines[0])
        position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}
        
#         {
#             'total_count' : 0,
#             'postings' : {
#                 'count':0,
#                 'doc_id':0,
#                 'positions':[]
#             }
#         }
        
        for line_no,line in enumerate(lines):
            doc_set = set()
            # split words at . , whitespace ? ! : ;
            position['row'] = line_no 
            position['col'] = 0
            for word in re.split('[.\s,?!:;-]', line):
                position['col'] += len(word) + 1
                position['token_no'] += 1
                # Case Folding
                word = word.lower()
                
                # Filter non-ASCII characters
                word = ''.join(filter(lambda x: x in printable, word))
                
                # Remove Punctuations
                word = remove_punctuation(word)
                
                if re.match('\d+[A-Za-z]+',word):
                    word = re.split('\d+',word)[1]
                if re.match('[A-Za-z]+\d+',word):
                    word = re.split('\d+',word)[0]
                
                if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                    continue
                if word in stop_words:
                    continue
                
                
                
                word = ps.stem(word)
                    
                vocab.add(word)
                
                doc_set.add(word)
                
                if word in inverted_index.index.keys():
                    
                    inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position)) 
                else:
                    plist = PostingList()
                    inverted_index.index[word] = plist
                    inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position))
                    
                
                
        
            
        
        inverted_index.docs[file_number] = doc_set
        doc_contents.append(doc_set)

In [72]:
# print('Total Vocabulary Size ')
# print(len(vocab))
# print('Total Number of Documents ')
# print(len(doc_contents))
# print(doc_contents[17])

len(inverted_index)
inverted_index.index[ps.stem('hammer')].occurrance



4850

{16: [{'doc': 16, 'row': 1, 'col': 7055, 'token_no': 1308},
  {'doc': 16, 'row': 1, 'col': 7326, 'token_no': 1361}],
 17: [{'doc': 17, 'row': 1, 'col': 3434, 'token_no': 649}],
 18: [{'doc': 18, 'row': 1, 'col': 1619, 'token_no': 315}],
 19: [{'doc': 19, 'row': 1, 'col': 26500, 'token_no': 5472},
  {'doc': 19, 'row': 1, 'col': 26542, 'token_no': 5480},
  {'doc': 19, 'row': 1, 'col': 26703, 'token_no': 5513},
  {'doc': 19, 'row': 1, 'col': 27855, 'token_no': 5757}],
 20: [{'doc': 20, 'row': 1, 'col': 8800, 'token_no': 1692}],
 21: [{'doc': 21, 'row': 1, 'col': 3743, 'token_no': 695}],
 24: [{'doc': 24, 'row': 1, 'col': 2185, 'token_no': 428}],
 25: [{'doc': 25, 'row': 1, 'col': 4633, 'token_no': 900}],
 27: [{'doc': 27, 'row': 1, 'col': 2989, 'token_no': 577}],
 33: [{'doc': 33, 'row': 1, 'col': 2072, 'token_no': 401}],
 34: [{'doc': 34, 'row': 1, 'col': 6415, 'token_no': 1206}],
 35: [{'doc': 35, 'row': 1, 'col': 2319, 'token_no': 437}],
 36: [{'doc': 36, 'row': 1, 'col': 5575, 'token_

# Boolean Model:

In [103]:
# print(sorted(list(vocab)))
# for index,doc in enumerate(doc_contents):
#     print('Vocab size of doc' + str(index))
#     print(len(doc))

vocab_list = sorted(list(vocab))

term_doc_matrix_np = np.zeros((len(vocab), len(doc_contents)))

for word_index, word in enumerate(vocab_list):
    word_row = []
    for doc_index, doc in enumerate(doc_contents):
        if word in doc:
            term_doc_matrix_np[word_index, doc_index] = 1
        else:
            term_doc_matrix_np[word_index, doc_index] = 0
            
print(term_doc_matrix_np)
    

[[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:

with open('pickled/vocab.p', 'ab') as vocab_file:
    pickle.dump(vocab, vocab_file)

In [24]:
with open('pickled/vocab.p', 'rb') as vocab_file:
    vocabf = pickle.load(vocab_file)

In [65]:
query = input('Enter your query : ')
query = ps.stem(query)
query_actions = []
query_wanted = []
if query in vocab:
    term_index = vocab_list.index(query)
    term_row = term_doc_matrix_np[term_index]
    print(term_row)
    doc_ids = np.argwhere(term_row == 1)
    print(doc_ids)

else:
    print(f'{query} not present in vocabulary')


Enter your query : running
[1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 0.]
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [24]
 [25]
 [26]
 [27]
 [28]
 [30]
 [32]
 [33]
 [34]
 [35]
 [36]
 [37]
 [39]
 [40]
 [41]
 [44]
 [45]
 [46]
 [47]
 [50]
 [51]
 [52]
 [53]]


In [66]:
test = '000dollar'
x = re.match('\d+[A-Za-z]+',test)
print(x)
x = re.split('\d+',test)
print(x)
# For matching queries like
# not hammer or pakistan
# (magnum or not hammer) or not (polish and pakistan)
x = re.match('(not)?\s*(\w+|(\((not)?\s*(\w+)\s+(and|or)\s+(not)?\s*(\w+)\)))\s+(or|and)\s+(not)?\s*(\w+|(\((not)?\s*(\w+)\s+(and|or)\s+(not)?\s*(\w+)\)))')



<re.Match object; span=(0, 9), match='000dollar'>
['', 'dollar']


In [66]:
ans = {'0', '1', '10', '11', '12', '16', '17', '18', '19', '2', '20', '21', '22', '24', '25', '26', '27', '28', '3', '30', '32', '33', '34', '35', '36', '37', '39', '4', '40', '41', '44', '45', '46', '47', '5', '50', '51', '52', '53', '6', '8', '9'}
ans2 =set([str(x[0]) for x in doc_ids])

In [67]:
print(len(ans))
print((ans2))
print(ans.difference(ans2))

42
{'22', '1', '53', '25', '37', '8', '24', '10', '0', '39', '50', '17', '35', '40', '21', '33', '27', '52', '3', '36', '20', '16', '26', '45', '9', '30', '2', '5', '12', '19', '4', '32', '28', '46', '18', '47', '41', '34', '11', '51', '6', '44'}
set()


# References :

http://www.pyregex.com/
http://cs231n.github.io/python-numpy-tutorial/

https://www.online-utility.org/text/analyzer.jsp

https://stackoverflow.com/questions/2118261/parse-boolean-arithmetic-including-parentheses-with-regex

https://regex101.com/r/M8z3U4/1

https://iq.opengenus.org/porter-stemmer/

https://unnikked.ga/how-to-build-a-boolean-expression-evaluator-518e9e068a65

https://ruslanspivak.com/lsbasi-part7/

In [44]:
query_actions = []
query_wanted = []
term_index = vocab_list.index(ps.stem('actions'))
term_row_actions = term_doc_matrix_np[term_index]

query_actions = np.argwhere(term_row_actions == 1)
term_index = vocab_list.index(ps.stem('wanted'))
term_row_wanted = term_doc_matrix_np[term_index]

query_wanted = np.argwhere(term_row_wanted == 1)

and_query = np.array([1 if x == 1 and y == 1 else 0 for x,y in zip(term_row_actions, term_row_wanted)])
and_doc_ids = np.argwhere(and_query == 1)
print(and_query)
print(and_doc_ids)

ans = {'37', '3', '19', '1', '9', '40', '51', '16', '15', '12', '31', '41', '39', '0', '53', '26', '29', '17', '24', '54', '7', '2', '5', '28', '42'}
ans2 =set([str(x[0]) for x in and_doc_ids])

print(len(ans))
print((ans2))
print(ans.difference(ans2))

[1 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0
 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0]
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 5]
 [ 7]
 [ 9]
 [12]
 [15]
 [16]
 [17]
 [19]
 [24]
 [26]
 [28]
 [29]
 [31]
 [37]
 [39]
 [40]
 [41]
 [42]
 [51]
 [53]
 [54]]
25
{'9', '19', '37', '53', '5', '42', '15', '2', '1', '26', '51', '24', '0', '39', '29', '31', '17', '41', '40', '28', '16', '12', '54', '3', '7'}
set()


In [102]:
from collections import deque 

line = 'box'
boperators = ['and', 'or']
uoperators = ['not']
  
stack = deque() 
        
def clean_word(word):
    # Case Folding
    word = word.lower()
     # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
#     print(word)
    # Remove Punctuations
    if word != '(' and word != ')':
        word = remove_punctuation(word)
#     print(word)
    if re.match('\d+[A-Za-z]+',word):
        word = re.split('\d+',word)[1]
    if re.match('[A-Za-z]+\d+',word):
        word = re.split('\d+',word)[0]
#     print(word)
    word = ps.stem(word)
#     print(word)
    return word
query = (re.split('[.\s,?!:;-]', line))


def evaluate_expression(index, query, stack, state):
    
    print("Current indexed Word : " + str(query[index]))
    result_query = []
    
    if query[index] == '(':
        bracket_term = query[index] 
        result, new_index = evaluate_expression(index+1, query, stack, state)
        if new_index == -1:
            state = []
        for x in range(index, new_index+1):
            state.pop(x)
            
        print("Result")
        print(result)
        index = new_index
        stack.append({'state':True,'data':result, 'query': 'not' + not_of_term})
        result_query = result
        
        print('new_index ' + str(new_index))
        print(f'This should be a ) = {query[new_index]}')
        
    
    
    
    
    
    
    
    if query[index] in uoperators:
        not_of_term = query[index] 
        result, new_index = evaluate_expression(index+1, query, stack, state)
        if new_index == -1:
            state = []
        for x in range(0, new_index-index+1):
            state.pop(x)
            
        print("Result")
        print(result)
        index = new_index
        not_result = [0 if int(x)==1 else 1 for x in result]
        stack.append({'state':True,'data':not_result, 'query': 'not' + not_of_term})
        result_query = not_result
        
    print(query[index])
    if len(state) == 0:
        return result_query, index
    print('STATE')
    print(state)
    
    
    if (query[index] not in boperators) and (query[index] not in uoperators):
        query[index] = ps.stem(query[index])
        if query[index] not in vocab_list:
            print(f'{query[index]} is not in vocabulary of index')
            return [], -1
        term_index = vocab_list.index(query[index])
        term_row = term_doc_matrix_np[term_index]
        stack.append({'state':True,'data':term_row,'query':query[index]})
        result_query = term_row
        index += 1
        
    if index >= len(query):
        return stack.pop()['data'], -1
    if len(state) == 0:
        return result_query, index
    
    elif query[index] in boperators:
#         query2 = clean_word(next_word)
        
#         next_word = query[index+1]
#         print(next_word)
#         if query2 not in vocab_list:
#             print(f'{query2} is not in vocabulary of index')
#             return [], -1
        
        
#         term_index2 = vocab_list.index(query2)
#         term_row2  = term_doc_matrix_np[term_index2]
        
        
        term_row2, new_index = evaluate_expression(index+1, query, stack, state)
        if new_index == -1:
            state = []
        for x in range(0, new_index-index+1):
            state.pop(x)
        
        
        query1 =  stack.pop()
        term_row1 = []
        
        if query1['state'] == False:
            term_index1 = vocab_list.index(query1['data'])
            term_row1 = term_doc_matrix_np[term_index1]
        
        else:
            term_row1 = query1['data']
        
        print(term_row1)
        
        print(term_row2)
        result_query = []
        if query[index] == 'and':
            and_query = np.array([1 if int(x) == 1 and int(y) == 1 else 0 for x,y in zip(term_row1, term_row2)])
            print(and_query)
            and_doc_ids = np.argwhere(and_query == 1)
            query_ans =  set([x[0] for x in and_doc_ids])
            print(query_ans)
            # stack.append({'state':True,'data':and_query,'query':query2})
            result_query = and_query
            
        elif query[index] == 'or':
            or_query = np.array([1 if int(x) == 1 or int(y) == 1 else 0 for x,y in zip(term_row1, term_row2)])
            print('OR')
            print(or_query)
            or_doc_ids = np.argwhere(or_query == 1)
            query_ans =  set([x[0] for x in or_doc_ids])
            print(query_ans)
            # stack.append({'state':True,'data':or_query,'query':query2})
            result_query = or_query
        index = new_index
        
    return result_query, index


# for index, word in enumerate(query):  
#     word = clean_word(word)
#     print(word)

query = [clean_word(word) for word in query ]
print(query)
ans, index = evaluate_expression(0, query, stack, query)

print("Outt")
print(list(ans))

print(stack)
final_doc_ids = np.argwhere(np.array(ans) == 1)
final_ans =  set([x[0] for x in final_doc_ids])
print(final_ans)

['box']
Current indexed Word : box
box
STATE
['box']
Outt
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
deque([])
{4, 9, 44, 45, 46, 47, 18, 50, 53, 54, 23, 25}


In [179]:
ansq ={'18', '46', '4', '45', '50', '9', '47', '23', '54', '44', '25'}
ansi = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}
ansq2 = set([str(x) for x in ansi])
print(ansq2)
ansq.difference(ansq2)

{'47', '22', '13', '27', '7', '38', '25', '42', '15', '4', '33', '6', '8', '34', '30', '21', '43', '55', '44', '50', '10', '29', '31', '11', '49', '18', '46', '23', '52', '45', '32', '36', '14', '54', '35', '20', '48'}


{'9'}

In [105]:
""" SPI - Simple Pascal Interpreter """
import re
boperators = ['and', 'or']
uoperators = ['not']

# Token types
#
# EOF (end-of-file) token is used to indicate that
# there is no more input left for lexical analysis
INTEGER, PLUS, MINUS, MUL, DIV, LPAREN, RPAREN, EOF, TERM, AND, OR, NOT = (
    'INTEGER', 'PLUS', 'MINUS', 'MUL', 'DIV', '(', ')', 'EOF', 'TERM', 'AND','OR', 'NOT'
)


class Token(object):
    def __init__(self, type, value):
        self.type = type
        self.value = value
        self.inverse = False
        self.row = []

    def __str__(self):
        """String representation of the class instance.

        Examples:
            Token(TERM, Hello)
            Token(AND, '&')
            Token(NOT, '!')
        """
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()


class Lexer(object):
    def __init__(self, text):
        # client string input, e.g. "hello | world & (why | are | you)"
        self.text = text
        # self.pos is an index into self.text
        self.pos = 0
        self.current_char = self.text[self.pos]
        

    def error(self):
        raise Exception('Invalid character')

    def advance(self):
        """Advance the `pos` pointer and set the `current_char` variable."""
        self.pos += 1
        if self.pos > len(self.text) - 1:
            self.current_char = None  # Indicates end of input
        else:
            self.current_char = self.text[self.pos]

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):
        """Return a (multidigit) integer consumed from the input."""
        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)
    def word(self):
        """Return a (multidigit) integer consumed from the input."""
        result = ''
        while self.current_char is not None and (self.current_char.isalpha() or self.current_char == '_'):
            result += self.current_char
            self.advance()
        return str(result)

    def get_next_token(self):
        """Lexical analyzer (also known as scanner or tokenizer)

        This method is responsible for breaking a sentence
        apart into tokens. One token at a time.
        """
        while self.current_char is not None:

            if self.current_char.isspace():
                self.skip_whitespace()
                continue

            if self.current_char.isdigit():
                return Token(INTEGER, self.integer())

            
            
            if self.current_char == '&':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(AND, 'AND')
            
            if self.current_char == '|':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(OR, 'OR')
            
            if self.current_char == '!':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(NOT,'NOT')
            
            if self.current_char == '+':
                self.advance()
                return Token(PLUS, '+')

            if self.current_char == '-':
                self.advance()
                return Token(MINUS, '-')

            if self.current_char == '*':
                self.advance()
                return Token(MUL, '*')

            if self.current_char == '/':
                self.advance()
                return Token(DIV, '/')

            if self.current_char == '(':
                self.advance()
                return Token(LPAREN, '(')

            if self.current_char == ')':
                self.advance()
                return Token(RPAREN, ')')
            
            if self.current_char.isalpha():      
#                 print('Got token  ' + self.current_char)
                return Token(TERM, self.word())
            
            

            self.error()

        return Token(EOF, None)


###############################################################################
#                                                                             #
#  PARSER                                                                     #
#                                                                             #
###############################################################################

class AST(object):
    pass


class BinOp(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right
        self.inverse = False
        self.row = []
        self.value = ''
        
class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
        self.inverse = False
        self.row = []

class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        # set current token to the first token taken from the input
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        # compare the current token type with the passed token
        # type and if they match then "eat" the current token
        # and assign the next token to the self.current_token,
        # otherwise raise an exception.
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        """factor : INTEGER | LPAREN expr RPAREN"""
        token = self.current_token
        
        if token.type == INTEGER:
            self.eat(INTEGER)
            return Num(token)
        
        elif token.type == TERM:
            self.eat(TERM)
            return Num(token)
        
        elif token.type == NOT:
            self.eat(NOT)
            node = self.expr()
            node.inverse = True
            return node
            
        
        
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        """term : factor ((MUL | DIV) factor)*"""
        node = self.factor()

        while self.current_token.type in (MUL, DIV, AND):
            token = self.current_token
            if token.type == MUL:
                self.eat(MUL)
            elif token.type == DIV:
                self.eat(DIV)
            elif token.type == AND:
                self.eat(AND)
        
            

            node = BinOp(left=node, op=token, right=self.factor())

        return node

    def expr(self):
        """
        expr   : term ((PLUS | MINUS) term)*
        term   : factor ((MUL | DIV) factor)*
        factor : INTEGER | LPAREN expr RPAREN
        """
        node = self.term()

        while self.current_token.type in (PLUS, MINUS, OR):
            token = self.current_token
            if token.type == OR:
                self.eat(OR)
            elif token.type == PLUS:
                self.eat(PLUS)
            elif token.type == MINUS:
                self.eat(MINUS)
            node = BinOp(left=node, op=token, right=self.term())

        return node

    def parse(self):
        return self.expr()


###############################################################################
#                                                                             #
#  INTERPRETER                                                                #
#                                                                             #
###############################################################################

class NodeVisitor(object):
    def visit(self, node):
        print('Checking Node Name')
        
        method_name = 'visit_' + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        raise Exception('No visit_{} method'.format(type(node).__name__))


class Interpreter(NodeVisitor):
    def __init__(self, parser):
        self.parser = parser

    def visit_BinOp(self, node):
        print('Bin OP : ' )
        print(node.token)
        print(node.value)
        print(node.row)
        print(node.inverse)
        if node.op.type == PLUS:
            return str(self.visit(node.left)) + '+' +  str(self.visit(node.right))
        elif node.op.type == MINUS:
            return str(self.visit(node.left)) + '-' +  str(self.visit(node.right))
        elif node.op.type == MUL:
            return str(self.visit(node.left)) + '*' +  str(self.visit(node.right))
        elif node.op.type == DIV:
            return str(self.visit(node.left)) + '/' +  str(self.visit(node.right))
        
        
        
        
        elif node.op.type == AND:
            print('Node => ')
            print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = [1 if int(x)==0 else 0 for x in left.row]
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = [1 if int(x)==0 else 0 for x in left.right]
                right.inverse = False
            
            node.row = [1 if int(x) == 1 and int(y) == 1 else 0 for x,y in zip(left.row, right.row)]
            if node.inverse == True:
                node.row = [1 if int(x)==0 else 0 for x in node.row]
                node.inverse = False
            
            return node
        
        elif node.op.type == OR:
            print('Node => ')
            print(node)
            left = self.visit(node.left)   
            print((left.token))
            print((left.row))
            
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = [1 if int(x)==0 else 0 for x in left.row]
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = [1 if int(x)==0 else 0 for x in left.right]
                right.inverse = False
            
            node.row = [1 if int(x) == 1 or int(y) == 1 else 0 for x,y in zip(left.row, right.row)]
            
            return node
        

    def visit_Num(self, node):
        print('Num  : ' )
        print(node.token)
        print(node.value)
        print(node.row)
        print(node.inverse)
        
        node.value = node.value.split('_')[0]
        if ps.stem(node.value) in vocab_list:
            
            term_index = vocab_list.index(ps.stem(node.value))
            term_row = term_doc_matrix_np[term_index]
        else:
            term_row = [0 for x in range(0,len(term_doc_matrix_np[0]))]
            print('Term Row')
            print(term_row)
            
        node.row = term_row
        if node.inverse == True:
            node.row = [1 if int(x)==0 else 0 for x in term_row]
            node.inverse = False
        
        return node

    def interpret(self):
        tree = self.parser.parse()
        print(tree)
        return self.visit(tree)

' SPI - Simple Pascal Interpreter '

In [110]:
text = input('Enter your query : ')
text = text.replace('and','&')
text = text.replace('AND','&')
text = text.replace('or','|')
text = text.replace('OR','|')
text = text.replace('NOT', '!')
text = text.replace('not','!')
print(text)

lexer = Lexer(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
print(result.value)
print(result.row)

final_doc_ids = np.argwhere(np.array(result.row) == 1)
final_ans =  set([x[0] for x in final_doc_ids])
print(final_ans)

Enter your query : pakistan OR afghanistan OR aid
pakistan | afghanistan | aid
Checking Node Name
Bin OP : 
Token(OR, 'OR')

[]
False
Node => 
Checking Node Name
Bin OP : 
Token(OR, 'OR')

[]
False
Node => 
Checking Node Name
Num  : 
Token(TERM, 'pakistan')
pakistan
[]
False
Token(TERM, 'pakistan')
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Checking Node Name
Num  : 
Token(TERM, 'afghanistan')
afghanistan
[]
False
Token(OR, 'OR')
[0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Checking Node Name
Num  : 
Token(TERM, 'aid')
aid
[]
False

[0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
{1, 3, 4, 37, 39, 40, 9, 41, 42, 16, 17

In [113]:
ansq ={'29', '16', '4', '22', '37', '40', '42', '18', '1', '17', '41', '39', '9', '3'}
# ansi = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}
ansq2 = set([str(x) for x in final_ans])
print(ansq2)
ansq.difference(ansq2)

{'3', '39', '29', '42', '17', '37', '9', '1', '41', '4', '22', '18', '40', '16'}


set()

In [104]:
b= clean_word('plane_set')
print(b)
ps.stem(b)

planeset


'planeset'

In [36]:
def intersect_posting(p1, p2):
    return p1.occurrance.keys() & p2.occurrance.keys()
def union_posting(p1, p2):
    return p1.occurrance.keys() | p2.occurrance.keys()
def inverse_posting(inverted_index,p):
    return inverted_index.docs.keys() - p.occurrance.keys()



In [43]:
final_ans = inverse_posting(inverted_index, inverted_index.index[ps.stem('hammer')])

In [44]:
ansq ={'31', '28', '37', '30', '7', '10', '14', '1', '6', '41', '15', '11', '29', '26', '52', '13', '32', '44', '4', '8', '22', '38', '48', '0', '47', '2', '23', '9', '3', '5', '12', '55'}
# ansi = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}
ansq2 = set([str(x) for x in final_ans])
print(ansq2)
ansq.difference(ansq2)

{'1', '37', '52', '31', '3', '14', '8', '22', '38', '4', '15', '41', '55', '10', '44', '0', '29', '2', '23', '48', '11', '12', '5', '7', '9', '30', '13', '32', '47', '28', '6', '26'}


set()

In [87]:
def positional_intersect(p1, p2, k):
    ip = intersect_posting(p1, p2)
    lip = sorted(list(ip))
    ans = []
    for doc in lip:
#         print(type(p1))
        positions1 = p1.occurrance[doc]
        positions2 = p2.occurrance[doc]
        index_p2 = 0
        index_p1 = 0
        for pos1 in positions1:
            for pos2 in positions2:
                if pos2['token_no'] -  pos1['token_no'] <= k and pos2['token_no'] -  pos1['token_no'] > 0:
                    ans.append({'doc':doc, 'pos1':  pos1, 'pos2':pos2})
            
        
        
    return ans
        

In [99]:
ans = positional_intersect(inverted_index.index[ps.stem('keep')], inverted_index.index[ps.stem('out')], 3)
final_ans = set(map(lambda pos: pos['doc'],ans))
ansq ={'20', '24', '39', '40', '51'}
# ansi = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}
ansq2 = set([str(x) for x in final_ans])
print(ansq2)
ansq.difference(ansq2)

{'40', '16', '24', '39', '12', '27', '20', '51', '26', '14', '8', '46', '45'}


set()

In [107]:
inverted_index.index[ps.stem('hillary')]
inverted_index.index[ps.stem('clinton')].occurrance[2]

total_cnt : 687 docs : [dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55])]

[{'doc': 2, 'row': 1, 'col': 7683, 'token_no': 1508},
 {'doc': 2, 'row': 1, 'col': 8050, 'token_no': 1576},
 {'doc': 2, 'row': 1, 'col': 8178, 'token_no': 1599},
 {'doc': 2, 'row': 1, 'col': 8375, 'token_no': 1632},
 {'doc': 2, 'row': 1, 'col': 8425, 'token_no': 1642},
 {'doc': 2, 'row': 1, 'col': 15804, 'token_no': 3106},
 {'doc': 2, 'row': 1, 'col': 16159, 'token_no': 3172}]

In [86]:
inverted_index.index[ps.stem('hillary')].occurrance[1]
inverted_index.index[ps.stem('clinton')].occurrance[1]

[{'doc': 1, 'row': 1, 'col': 11216, 'token_no': 2225}]

[{'doc': 1, 'row': 1, 'col': 11224, 'token_no': 2226}]

In [122]:
""" SPI - Simple Pascal Interpreter """
import re
boperators = ['and', 'or']
uoperators = ['not']

# Token types
#
# EOF (end-of-file) token is used to indicate that
# there is no more input left for lexical analysis
LPAREN, RPAREN, EOF, TERM, AND, OR, NOT = (
    '(', ')', 'EOF', 'TERM', 'AND','OR', 'NOT'
)


class Token(object):
    def __init__(self, type, value):
        self.type = type
        self.value = value
        self.inverse = False
        self.row = []

    def __str__(self):
        """String representation of the class instance.

        Examples:
            Token(TERM, Hello)
            Token(AND, '&')
            Token(NOT, '!')
        """
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()


class Lexer(object):
    def __init__(self, text):
        # client string input, e.g. "hello | world & (why | are | you)"
        self.text = text
        # self.pos is an index into self.text
        self.pos = 0
        self.current_char = self.text[self.pos]
        

    def error(self):
        raise Exception('Invalid character')

    def advance(self):
        """Advance the `pos` pointer and set the `current_char` variable."""
        self.pos += 1
        if self.pos > len(self.text) - 1:
            self.current_char = None  # Indicates end of input
        else:
            self.current_char = self.text[self.pos]

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):
        """Return a (multidigit) integer consumed from the input."""
        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)
    def word(self):
        """Return a (multidigit) integer consumed from the input."""
        result = ''
        while self.current_char is not None and (self.current_char.isalpha() or self.current_char == '_'):
            result += self.current_char
            self.advance()
        return str(result)

    def get_next_token(self):
        """Lexical analyzer (also known as scanner or tokenizer)

        This method is responsible for breaking a sentence
        apart into tokens. One token at a time.
        """
        while self.current_char is not None:

            if self.current_char.isspace():
                self.skip_whitespace()
                continue

            if self.current_char.isdigit():
                return Token(INTEGER, self.integer())

            
            
            if self.current_char == '&':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(AND, 'AND')
            
            if self.current_char == '|':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(OR, 'OR')
            
            if self.current_char == '!':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(NOT,'NOT')
            

            if self.current_char == '(':
                self.advance()
                return Token(LPAREN, '(')

            if self.current_char == ')':
                self.advance()
                return Token(RPAREN, ')')
            
            if self.current_char.isalpha():      
#                 print('Got token  ' + self.current_char)
                return Token(TERM, self.word())
            
            

            self.error()

        return Token(EOF, None)


###############################################################################
#                                                                             #
#  PARSER                                                                     #
#                                                                             #
###############################################################################

class AST(object):
    pass


class BinOp(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right
        self.inverse = False
        self.row = []
        self.value = ''
        
class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
        self.inverse = False
        self.row = []

class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        # set current token to the first token taken from the input
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        # compare the current token type with the passed token
        # type and if they match then "eat" the current token
        # and assign the next token to the self.current_token,
        # otherwise raise an exception.
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        """factor : INTEGER | LPAREN expr RPAREN"""
        token = self.current_token
        
        
        if token.type == TERM:
            self.eat(TERM)
            return Num(token)
        
        elif token.type == NOT:
            self.eat(NOT)
            node = self.expr()
            node.inverse = True
            return node
            
        
        
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        """term : factor ((MUL | DIV) factor)*"""
        node = self.factor()

        while self.current_token.type in (AND,):
            token = self.current_token
         
            if token.type == AND:
                self.eat(AND)
        
            

            node = BinOp(left=node, op=token, right=self.factor())

        return node

    def expr(self):
        """
        expr   : term ((PLUS | MINUS) term)*
        term   : factor ((MUL | DIV) factor)*
        factor : INTEGER | LPAREN expr RPAREN
        """
        node = self.term()

        while self.current_token.type in (OR,):
            token = self.current_token
            if token.type == OR:
                self.eat(OR)
            node = BinOp(left=node, op=token, right=self.term())

        return node

    def parse(self):
        return self.expr()


class NodeVisitor(object):
    def visit(self, node):
        print('Checking Node Name')
        
        method_name = 'visit_' + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        raise Exception('No visit_{} method'.format(type(node).__name__))


class Interpreter(NodeVisitor):
    def __init__(self, parser, index, ps):
        self.parser = parser
        self.index = index
        self.ps = ps

    def visit_BinOp(self, node):
        print('Bin OP : ' )
        print(node.token)
        print(node.value)
        print(node.row)
        print(node.inverse)
        if node.op.type == AND:
            print('Node => ')
            print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = self.index.index[self.ps.stem(left.row)]
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = self.index.index[self.ps.stem(right.row)]
                right.inverse = False
            
            node.row = intersect_posting(left.row, right.row)
            if node.inverse == True:
                node.row = self.index.index[self.ps.stem(node.row)]
                node.inverse = False
            
            return node
        
        elif node.op.type == OR:
            print('Node => ')
            print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = self.index.index[self.ps.stem(left.row)]
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = self.index.index[self.ps.stem(right.row)]
                right.inverse = False
            
            node.row = union_posting(left.row, right.row)
            if node.inverse == True:
                node.row = self.index.index[self.ps.stem(node.row)]
                node.inverse = False
            
            return node
        

    def visit_Num(self, node):
        print('Num  : ' )
        print(node.token)
        print(node.value)
        print(node.row)
        print(node.inverse)
        
        node.value = node.value.split('_')[0]
        if self.ps.stem(node.value) in self.index.index.keys():
            
            term_docs = self.index.index[self.ps.stem(node.value)]
            
        else:
            term_docs = {}
            print('Term Row')
            print(term_docs)
            
        node.row = term_docs
        if node.inverse == True:
            node.row = inverse_posting(node.row)
            node.inverse = False
        
        return node

    def interpret(self):
        tree = self.parser.parse()
        print(tree)
        return self.visit(tree)

' SPI - Simple Pascal Interpreter '

In [125]:
text = input('Enter your query : ')
text = text.replace('and','&')
text = text.replace('AND','&')
text = text.replace('or','|')
text = text.replace('OR','|')
text = text.replace('NOT', '!')
text = text.replace('not','!')
print(text)

lexer = Lexer(text)
parser = Parser(lexer)
interpreter = Interpreter(parser, inverted_index, ps)
result = interpreter.interpret()
print(result.value)
print(result.row)

# final_doc_ids = np.argwhere(np.array(result.row) == 1)
# final_ans =  set([x[0] for x in result.row])
# print(final_ans)

Enter your query : united or plane
united | plane
Checking Node Name
Bin OP : 
Token(OR, 'OR')

[]
False
Node => 
Checking Node Name
Num  : 
Token(TERM, 'united')
united
[]
False
Checking Node Name
Num  : 
Token(TERM, 'plane')
plane
[]
False

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}


In [126]:
ans = result.row
final_ans = ans
# final_ans = set(map(lambda pos: pos['doc'],ans))
ansq ={'31', '28', '50', '46', '37', '30', '54', '10', '18', '7', '1', '17', '41', '49', '6', '34', '36', '11', '45', '29', '26', '52', '13', '21', '24', '16', '25', '32', '33', '4', '44', '22', '8', '19', '40', '20', '38', '48', '0', '47', '27', '51', '43', '2', '35', '39', '9', '3', '5', '12', '55'}
# ansi = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}
ansq2 = set([str(x) for x in final_ans])
print(ansq2)
ansq.difference(ansq2)

{'1', '37', '39', '35', '20', '31', '52', '3', '22', '8', '38', '45', '4', '21', '41', '19', '55', '10', '44', '0', '29', '16', '2', '40', '50', '34', '48', '24', '11', '12', '5', '7', '27', '33', '36', '54', '43', '46', '9', '30', '13', '32', '47', '17', '26', '28', '6', '51', '18', '25', '49'}


set()